<a href="https://colab.research.google.com/github/Chry593/Data-science/blob/main/Pulire_not_null_values.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Cleaning Not Null Values**
Certe volte possiamo trovare dei valori invalidi. Per esempio:

In [2]:
import numpy as np
import pandas as pd

In [24]:
df = pd.DataFrame({
    "Sex" : ["M","F","F","D","?"],
    "Age" : [29,30,24,290,25],
})
df

,Sex,Age
0,M,29
1,F,30
2,F,24
3,D,290
4,?,25


Questo dataframe ad esempio non ha valori mancanti ma ha valori invalidi, tipo "**D**" e "**?**" o anche l'eta "**290**"

### **Finding Unique Values**
Il primo step per identificare i valori è identificarli e sistemarli (rimpiazzare,rimuoverli,ecc). Normalmente, per le categorie (come Sex, che contiene solo due valore (M,F)) iniziamo ad analizzar la varietà dei valori presenti. Per questo usiamo il metodo unique():

In [5]:
df["Sex"].unique()

array(['M', 'F', 'D', '?'], dtype=object)

Ad esempio ora possiamo notare D e ? che non c'entrano nulla con il sesso, quindi dobbiamo rimpiazzarli:

In [6]:
df["Sex"].replace("D", "F")

0    M
1    F
2    F
3    F
4    ?
Name: Sex, dtype: object

Oppure possiamo usare un dizionario:

In [20]:
df["Sex"].replace({"D":"F", "N":"M"})

0    M
1    F
2    F
3    F
4    ?
Name: Sex, dtype: object

Se abbiamo molte colonne da rimpiazzare, possiamo applicare un DataFrame level:

In [9]:
df.replace({
    "Sex" : {
        "D" : "F",
        "N" : "M"
    },
    "Age" : {
        290 : 29
    }
})

,Sex,Age
0,M,29
1,F,30
2,F,24
3,F,29
4,?,25


Abbiamo visto che 290 è un'età invalida per gli essere umani, quindi andremo a levare tutti gli 0 in eccesso all'età:

In [25]:
df[df["Age"] > 100]

,Sex,Age
3,D,290


In [31]:
df.loc[df["Age"] > 100,"Age"] = df.loc[df["Age"] > 100, "Age"] / 10   #righe con eta maggiore 100,prendi solo eta
df

,Sex,Age
0,M,29
1,F,30
2,F,24
3,D,29
4,?,25


### **Duplicates**
Controllare i duplicati è molto semplice. Iniziamo con le serie:

In [34]:
ambassadors = pd.Series([
    "France",
    "United Kingdom",
    "United Kingdom",
    "Italy",
    "Germany",
    "Germany",
    "Germany",
], index = [
    "Gérard Araud",
    "Kim Darroch",
    "Peter Wastmacott",
    "Armando Varricchio",
    "Peter Witting",
    "Peter Ammon",
    "Klaus Scharioth",
])
ambassadors

Gérard Araud                  France
Kim Darroch           United Kingdom
Peter Wastmacott      United Kingdom
Armando Varricchio             Italy
Peter Witting                Germany
Peter Ammon                  Germany
Klaus Scharioth              Germany
dtype: object

I due più importanti metodi per i duplicati sono *duplicated* (che ci dice quali valori sono duplicati) e drop_duplicates(ci da la riga dei duplicati)

In [36]:
ambassadors.duplicated()

Gérard Araud          False
Kim Darroch           False
Peter Wastmacott       True
Armando Varricchio    False
Peter Witting         False
Peter Ammon            True
Klaus Scharioth        True
dtype: bool

In questo caso duplicated non conta Kim Darroch , ma Peter Wastmacott che è la prima occorenza. Possiamo cambiare questa cosa mettendo il parametro **keep**:

In [37]:
ambassadors.duplicated(keep="last")

Gérard Araud          False
Kim Darroch            True
Peter Wastmacott      False
Armando Varricchio    False
Peter Witting          True
Peter Ammon            True
Klaus Scharioth       False
dtype: bool

In questo caso controlla dal basso verso l'altro. Per vedere tutti i duplicati basta mettere **keep=False**

In [38]:
ambassadors.duplicated(keep=False)

Gérard Araud          False
Kim Darroch            True
Peter Wastmacott       True
Armando Varricchio    False
Peter Witting          True
Peter Ammon            True
Klaus Scharioth        True
dtype: bool

In [41]:
ambassadors.drop_duplicates()

Gérard Araud                  France
Kim Darroch           United Kingdom
Armando Varricchio             Italy
Peter Witting                Germany
dtype: object

In [42]:
ambassadors.drop_duplicates(keep=False)

Gérard Araud          France
Armando Varricchio     Italy
dtype: object

In [43]:
ambassadors.drop_duplicates(keep="last")

Gérard Araud                  France
Peter Wastmacott      United Kingdom
Armando Varricchio             Italy
Klaus Scharioth              Germany
dtype: object

### **Duplicates in DataFrame**

In [44]:
players = pd.DataFrame({
    "Name" : [
        "Kobe Bryant",
        "Lebron James",
        "Kobe Bryant",
        "Carmelo Anthony",
        "Kobe Bryant",
    ],
    "Pos" : [
        "SG",
        "SF",
        "SG",
        "SF",
        "SF"
    ]
})

In [45]:
players

,Name,Pos
0,Kobe Bryant,SG
1,Lebron James,SF
2,Kobe Bryant,SG
3,Carmelo Anthony,SF
4,Kobe Bryant,SF


In [46]:
players.duplicated()

0    False
1    False
2     True
3    False
4    False
dtype: bool

Ci da solo la posizione due True poichè la posizione 0 e la 2 hanno lo stesso "Pos", quindi possiamo andare a customizzare il paramentro **subset**:

In [47]:
players.duplicated(subset = ["Name"])

0    False
1    False
2     True
3    False
4     True
dtype: bool

In [48]:
players.duplicated(subset = ["Name"], keep = "last")

0     True
1    False
2     True
3    False
4    False
dtype: bool

In [49]:
players.drop_duplicates()

,Name,Pos
0,Kobe Bryant,SG
1,Lebron James,SF
3,Carmelo Anthony,SF
4,Kobe Bryant,SF


In [50]:
players.drop_duplicates(subset = ["Name"])

,Name,Pos
0,Kobe Bryant,SG
1,Lebron James,SF
3,Carmelo Anthony,SF


In [52]:
players.drop_duplicates(subset = ["Name"], keep = "last")

,Name,Pos
1,Lebron James,SF
3,Carmelo Anthony,SF
4,Kobe Bryant,SF


### **Text Handing**
Pulire un testo può essere veramente difficile. Un testo invalido al 99% delle volte, non può essere previsto e non segue nessuna logica. Per fortuna ai giorni d'oggi non è così comune. Vediamo come risolvere:

**Splitting Columns**

In [107]:
df = pd.DataFrame({
    "Data" : [
        "1987_M_US _1",
        "1990?_M_UK_1",
        "1992_F_US_2",
        "1970?_M_   IT_1",
        "1985_F_I  T_2"
    ]
})

In [80]:
df

,Data
0,1987_M_US _1
1,1990?_M_UK_1
2,1992_F_US_2
3,1970?_M_ IT_1
4,1985_F_I T_2


Sappiamo che le singole colonne rappresentano "anno,sesso,paese e quanti figli hanno", ma sono tutte raggruppate nella stessa colonna e separati da underscore. Pandas ha un metodo chiamato **split** che è possibile usare in queste situazioni:

In [108]:
df["Data"].str.split("_")

0       [1987, M, US , 1]
1       [1990?, M, UK, 1]
2        [1992, F, US, 2]
3    [1970?, M,    IT, 1]
4      [1985, F, I  T, 2]
Name: Data, dtype: object

In [109]:
df["Data"].str.split("_", expand=True)

,0,1,2,3
0,1987,M,US,1
1,1990?,M,UK,1
2,1992,F,US,2
3,1970?,M,IT,1
4,1985,F,I T,2


In [110]:
df = df["Data"].str.split("_", expand=True)
df

,0,1,2,3
0,1987,M,US,1
1,1990?,M,UK,1
2,1992,F,US,2
3,1970?,M,IT,1
4,1985,F,I T,2


In [113]:
df.columns = ["Data","Sex","Country","Children"]
df

,Data,Sex,Country,Children
0,1987,M,US,1
1,1990?,M,UK,1
2,1992,F,US,2
3,1970?,M,IT,1
4,1985,F,I T,2


In [ ]:
df["Data"] = df["Data"].str.replace("?","")

In [116]:
df

,Data,Sex,Country,Children
0,1987,M,US,1
1,1990,M,UK,1
2,1992,F,US,2
3,1970,M,IT,1
4,1985,F,I T,2


### **Riassunto comandi**

In [117]:
df["Sex"].unique() #ci fa vedere nomi

array(['M', 'F'], dtype=object)

In [ ]:
df["Sex"].replace(vecchio,nuovo) #rimpiazza

In [ ]:
df["Sex"].replace({"D":"F", "N":"M"}) #rimpiazzamento a dizioanrio

In [118]:
df.replace({
    "Sex" : {
        "D" : "F",
        "N" : "M"
    },
    "Age" : {
        290 : 29
    }
}) #Se abbiamo molte colonne da rimpiazzare, possiamo applicare un DataFrame level

,Data,Sex,Country,Children
0,1987,M,US,1
1,1990,M,UK,1
2,1992,F,US,2
3,1970,M,IT,1
4,1985,F,I T,2


In [ ]:
df.loc[df["Age"] > 100,"Age"] = df.loc[df["Age"] > 100, "Age"] / 10   #righe con eta maggiore 100,prendi solo eta

In [119]:
ambassadors.duplicated() #True per la prima occorenza del duplicato

Gérard Araud          False
Kim Darroch           False
Peter Wastmacott       True
Armando Varricchio    False
Peter Witting         False
Peter Ammon            True
Klaus Scharioth        True
dtype: bool

In [120]:
ambassadors.duplicated(keep="last") #basso verso alto

Gérard Araud          False
Kim Darroch            True
Peter Wastmacott      False
Armando Varricchio    False
Peter Witting          True
Peter Ammon            True
Klaus Scharioth       False
dtype: bool

In [121]:
ambassadors.duplicated(keep=False) #tutti i duplicati

Gérard Araud          False
Kim Darroch            True
Peter Wastmacott       True
Armando Varricchio    False
Peter Witting          True
Peter Ammon            True
Klaus Scharioth        True
dtype: bool

In [122]:
ambassadors.drop_duplicates()  #droppa i duplicati lasciandodi i non

Gérard Araud                  France
Kim Darroch           United Kingdom
Armando Varricchio             Italy
Peter Witting                Germany
dtype: object

In [123]:
ambassadors.drop_duplicates(keep="last") #droppa basso verso l'alto

Gérard Araud                  France
Peter Wastmacott      United Kingdom
Armando Varricchio             Italy
Klaus Scharioth              Germany
dtype: object

In [125]:
ambassadors.drop_duplicates(keep=False) #droppa tutti duplicati lasciando i non

Gérard Araud          France
Armando Varricchio     Italy
dtype: object

In [126]:
players.duplicated(subset = ["Name"]) #duplicati dataframe per nome si può usare keep anche qui

0    False
1    False
2     True
3    False
4     True
dtype: bool

In [129]:
players.drop_duplicates(subset = ["Name"]) #duplicati dataframe per nome si può usare keep anche qui

,Name,Pos
0,Kobe Bryant,SG
1,Lebron James,SF
3,Carmelo Anthony,SF


Formattazione testo

In [142]:
df = pd.DataFrame({
    "Data" : [
        "1987_M_US _1",
        "1990?_M_UK_1",
        "1992_F_US_2",
        "1970?_M_   IT_1",
        "1985_F_I  T_2"
    ]
})

In [131]:
df["Data"].str.split("_")

0       [1987, M, US , 1]
1       [1990?, M, UK, 1]
2        [1992, F, US, 2]
3    [1970?, M,    IT, 1]
4      [1985, F, I  T, 2]
Name: Data, dtype: object

In [143]:
df = df["Data"].str.split("_", expand=True)
df

,0,1,2,3
0,1987,M,US,1
1,1990?,M,UK,1
2,1992,F,US,2
3,1970?,M,IT,1
4,1985,F,I T,2


In [144]:
df.columns = ["Data","Sex","Country","Children"]
df

,Data,Sex,Country,Children
0,1987,M,US,1
1,1990?,M,UK,1
2,1992,F,US,2
3,1970?,M,IT,1
4,1985,F,I T,2


In [146]:
df["Data"] = df["Data"].str.replace("?","")
df

<ipython-input-146-c06013cf9bce>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["Data"] = df["Data"].str.replace("?","")


,Data,Sex,Country,Children
0,1987,M,US,1
1,1990,M,UK,1
2,1992,F,US,2
3,1970,M,IT,1
4,1985,F,I T,2
